In [2]:
!pip install pdfplumber

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 24.5 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 60.4 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import fitz  # PyMuPDF
import pandas as pd
import os

# KRX보고서 폴더 경로 설정
base_folder_path = '/home/elicer/KRX_LLM/규정및 시행규칙'

# PDF 파일 제목과 내용을 저장할 리스트
pdf_data = []

# KRX보고서 폴더 내 모든 하위 폴더 순회
for folder_name in os.listdir(base_folder_path):
    folder_path = os.path.join(base_folder_path, folder_name)
    if os.path.isdir(folder_path):  # 하위 폴더인지 확인
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.pdf'):
                file_path = os.path.join(folder_path, file_name)
                try:
                    pdf_document = fitz.open(file_path)
                    pdf_text = ""
                    # 페이지별로 처리
                    for page_num in range(pdf_document.page_count):
                        try:
                            page = pdf_document.load_page(page_num)
                            pdf_text += page.get_text()  # 각 페이지의 텍스트를 추가
                        except Exception as e:
                            print(f"페이지 {page_num}에서 오류 발생: {e}")
                    pdf_document.close()
                    # 파일 제목과 내용을 리스트에 저장
                    pdf_data.append([file_name, pdf_text])
                except Exception as e:
                    print(f"파일 {file_name}을(를) 처리하는 동안 오류 발생: {e}")

# PDF 제목과 내용을 데이터프레임에 저장
df = pd.DataFrame(pdf_data, columns=['Title', 'Context'])

df.to_csv('rules.csv', index=False)
# 데이터프레임 출력
print(len(df))


TypeError: NDFrame.to_csv() got an unexpected keyword argument 'ignore_index'

In [2]:
df.to_csv('rules.csv', index=False, escapechar='\\')

In [3]:
# 문자열 데이터의 평균 길이 계산
average_length = df['Context'].apply(len).mean()
print("평균 문자열 길이:", average_length)

평균 문자열 길이: 42055.30136986302


In [11]:
import fitz  # PyMuPDF
import re
from transformers import pipeline
import pandas as pd

# 1. PDF 파일에서 텍스트 추출
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf[page_num]
            text += page.get_text()
    return text

# 2. 텍스트 전처리 (문단 나누기 및 정제)
def preprocess_text(text):
    # "공백 + 줄바꿈" 패턴을 문단 구분 기준으로 설정
    paragraphs = re.split(r'(?<=\S)\s*\n\s*\n+', text)  # 공백 이후의 줄바꿈을 기준으로 문단 분리
    cleaned_paragraphs = []

    for para in paragraphs:
        # 각 문단 내부에서 줄바꿈 제거 및 하이픈 처리
        lines = para.split("\n")
        buffer = ""
        for line in lines:
            # 하이픈으로 끝나는 줄은 다음 줄과 연결
            if line.endswith("-"):
                buffer += line[:-1]  # 하이픈 제거하고 줄 연결
            else:
                buffer += line.strip() + " "  # 줄바꿈을 제거하고 띄어쓰기 추가
        
        # 완성된 문단을 하나의 문자열로 정리
        cleaned_paragraph = re.sub(r'\s+', ' ', buffer).strip()
        if cleaned_paragraph:
            cleaned_paragraphs.append(cleaned_paragraph)
    
    return cleaned_paragraphs


# 3. 질문 생성 및 답변 생성
def create_qa_pairs(paragraphs):
    question_generator = pipeline("text2text-generation", model="mrm8488/t5-base-finetuned-question-generation-ap")  # 질문 생성 모델
    answer_generator = pipeline("question-answering")  # 답변 생성 모델
    qa_pairs = []

    for para in paragraphs:
        try:
            # 각 문단에 대해 질문 생성
            question_result = question_generator(f"generate question: {para}")
            question = question_result[0]['generated_text']
            
            # 생성된 질문에 대해 답변 생성
            answer_result = answer_generator(question=question, context=para)
            answer = answer_result['answer']

            # QA 쌍 추가
            qa_pairs.append({
                "question": question,
                "answer": answer
            })
        except Exception as e:
            print(f"질문 생성 중 오류 발생: {e}")
    return qa_pairs

# 4. 데이터프레임으로 변환 및 CSV 저장
def save_to_csv(qa_pairs, output_path):
    # DataFrame 생성
    df = pd.DataFrame(qa_pairs)
    # CSV 파일로 저장
    df.to_csv(output_path, index=False, encoding="utf-8-sig")

# 종합 코드 실행
pdf_path = "/home/elicer/KRX_LLM/규정및 시행규칙/새 폴더/거래증거금 및 청산증거금 현금 운용지침.pdf"  # PDF 파일 경로
output_path = "qa_dataset.csv"     # 저장할 CSV 파일 경로

# 단계별 수행
text = extract_text_from_pdf(pdf_path)      # PDF 파일에서 텍스트 추출
print(text)
paragraphs = preprocess_text(text)          # 텍스트 전처리 (문단 나누기)
print(len(paragraphs), paragraphs)
qa_pairs = create_qa_pairs(paragraphs)      # 질의응답 생성
save_to_csv(qa_pairs, output_path)          # CSV 파일로 저장

print("QA 데이터셋이 성공적으로 CSV 파일로 저장되었습니다.")


거래증거금및청산증거금현금운용지침
제1장총칙
제1조(목적) 이지침은「유가증권시장업무규정시행세칙」제99조의
24,
「코스닥시장
업무규정
시행세칙」
제35조의40,
「코넥스시장
업무규정
시행세칙」
제46조의12,
「파생상품시장
업무규정
시행세
칙」
제102조의2
및
「장외파생상품
청산업무규정
시행세칙」
제58
조에따라한국거래소가회원으로부터증거금으로예탁받은현금
을운용하는데필요한사항을정함을목적으로한다.
제2조(정의) ①이지침에서사용하는용어의정의는다음각호와같
다.
1.
“회원”이란
「회원관리규정」
제3조제2항제1호에
따른
결제회원
과
「장외파생상품
청산업무규정」
제2조제1항제3호에
따른
청산
회원을말한다.
2.
“증거금”이란
「유가증권시장
업무규정」
제72조의2제1항,
「코스
닥시장
업무규정」
제28조의2제1항,
「코넥스시장
업무규정」
제
34조제1항및「파생상품시장업무규정」제88조제1항에따른거
래증거금과
「장외파생상품
청산업무규정」
제81조제1항에
따른
청산증거금을말한다.
전부개정
2021.9.30 지침제749호
개
정
2021.12.27 지침제758호
개
정
2022.12.16 지침제805호
개
정
2023.3.24 지침제824호
개
정
2023.7.11 지침제844호
3. “증거금현금”이란제2호의증거금으로서한국거래소(이하“거래
소”라한다)가회원으로부터예탁받은현금(외화를포함한다)을
말한다.
4.
“결제은행”이란
「유가증권시장
업무규정」
제72조의5제1항,
「코
스닥시장
업무규정」
제28조의5제1항,
「코넥스시장
업무규정」
제34조의4제1항,
「파생상품시장
업무규정」
제91조제1항과
「장
외파생상품청산업무규정」제85조제1항에따라증거금을예탁받
기위해거래소명의의계좌를개설한은행을말한다.
5. “일반운용”이란거래소의청산결제업무를수행하는부서(이하
“관리부서”라한다)가증거금현금을운용하는것을말한다.
6. “특별운용”이란거래소의재무회계업무를수행하는부서(이하
“특별운용부서”라한다)가증거금현금을운용하는것을말한다.
7. “CCP리스크관리부서”란거래소의CCP리스크관리업무를담당하
는부서로서특별

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingfa

QA 데이터셋이 성공적으로 CSV 파일로 저장되었습니다.
